In [1]:
import numpy as np, scipy, matplotlib.pyplot as plt, IPython.display as ipd, sklearn
import librosa, librosa.display
import os
import warnings
import tqdm
import json
import pickle
import pandas as pd

warnings.filterwarnings('ignore')

In [2]:
# Retrieve the target based on the directories
SOUND_FOLDER = "./Audio_Dataset"
FEAT_FOLDER = "./Features"

In [3]:
# Utilities
def normalize(x, axis=0):
    return sklearn.preprocessing.minmax_scale(x, axis=axis)

def sigmoid_function(x):
    return 1.0/(1.0+np.exp(-x))


---
# **FEATURE EXTRACTION**




In [4]:
# Feature extraction
def extract_spectral_centroid(y,sr):
    spectral_centroid = librosa.feature.spectral_centroid(y=y+0.01, sr=sr)[0]
    return [spectral_centroid, float("{:.2f}".format(np.mean(spectral_centroid)))]

def extract_spectral_band_width(y,sr):
    spectral_band_width = librosa.feature.spectral_bandwidth(y=y+0.01, sr=sr, p=3)[0]
    return [spectral_band_width, float("{:.2f}".format(np.mean(spectral_band_width)))]

def extract_mfcc(y, sr):
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfcc_mean = np.mean(mfcc, axis=1)
    return mfcc, mfcc_mean

def extract_spectral_contrast(y, sr):
    contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    contrast_mean = np.mean(contrast, axis=1)
    return contrast, contrast_mean


In [5]:
path = os.path.join(SOUND_FOLDER)

In [6]:
# Retrieve list of files
list_of_files = []

# dirs=directories
for (root, dirs, file) in os.walk(path):
    for f in file:
        if '.wav' in f:
            list_of_files.append(root+"/"+f)

In [7]:
# Define the Dataframes of features
feat_df = pd.DataFrame(columns=['file_name'])

In [8]:
# -- FEATURE EXTRECTION --
# Create the dictionary list
#feat_dict = list()
#d = dict()

# Define the Dataframes of features
#feat_df = pd.DataFrame(columns=['file_name'])

for fname in tqdm.tqdm(list_of_files):
  #d['path'] = fname
  #d['file_name'] = os.path.basename(fname).split('/')[-1]
  signal, sr = librosa.load(path=fname)

  db_new_row = {'path': fname}
  db_new_row['file_name'] = os.path.basename(fname).split('/')[-1]

  # Spectral Centroid
  feat, feat_mean = extract_spectral_centroid(y=signal, sr=sr)
  #d['SC_mean'] = feat_mean
  db_new_row['SC_mean'] = feat_mean

  # Spectral BandWidth
  feat, feat_mean = extract_spectral_band_width(y=signal, sr=sr)
  #d['SBW_mean'] = feat_mean
  db_new_row['SBW_mean'] = feat_mean
  
  # MFCC
  feat, feat_mean = extract_mfcc(y=signal, sr=sr)
  #d['MFCC_mean'] = feat_mean.tolist()
  for i, val in enumerate(feat_mean):
    db_new_row["MFCC_mean_"+str(i)] = val

  # Spectral Contrast
  feat, feat_mean = extract_spectral_contrast(signal,sr)
  #d['Contrast_mean'] = feat_mean.tolist()
  for i, val in enumerate(feat_mean):
    db_new_row["Contrast_mean"+str(i)] = val

  #feat_dict.append(d.copy())
  s = pd.Series(db_new_row)
  feat_df = pd.concat([feat_df, s.to_frame().T], ignore_index=True)

  0%|          | 0/160 [00:00<?, ?it/s]

100%|██████████| 160/160 [00:06<00:00, 22.87it/s]


In [18]:
feat_df

,file_name,path,SC_mean,SBW_mean,MFCC_mean_0,MFCC_mean_1,MFCC_mean_2,MFCC_mean_3,MFCC_mean_4,MFCC_mean_5,...,MFCC_mean_10,MFCC_mean_11,MFCC_mean_12,Contrast_mean0,Contrast_mean1,Contrast_mean2,Contrast_mean3,Contrast_mean4,Contrast_mean5,Contrast_mean6
0,Bass Sample 40.wav,./Audio_Dataset/kick/Bass Sample 40.wav,150.75,1015.73,-532.827209,31.943409,17.710817,12.833717,9.849707,9.167212,...,3.357901,3.218079,3.006002,21.344141,14.500124,13.399614,15.123094,19.39248,19.998162,29.343212
1,Bass Sample 8.wav,./Audio_Dataset/kick/Bass Sample 8.wav,417.12,2382.5,-493.95929,35.429024,22.820415,27.60227,22.493084,20.092981,...,13.518476,11.085152,9.577057,15.871036,13.586046,14.402576,13.256011,14.459496,14.985597,42.308677
2,Bass Sample 23.wav,./Audio_Dataset/kick/Bass Sample 23.wav,197.32,1448.13,-537.407654,29.824848,18.296658,21.677267,16.876104,14.519206,...,5.045535,3.249687,2.544477,14.909034,17.027075,14.299427,14.1403,15.249436,16.00213,39.271722
3,Bass Sample 37.wav,./Audio_Dataset/kick/Bass Sample 37.wav,371.22,2010.63,-449.292969,26.058981,13.255217,18.491781,11.475266,12.965652,...,7.157709,7.861504,4.533995,20.283964,13.85492,13.226853,13.593988,15.003679,15.396403,44.395186
4,Bass Sample 36.wav,./Audio_Dataset/kick/Bass Sample 36.wav,326.61,1836.81,-462.241608,23.550018,11.211488,17.141399,9.932425,11.088989,...,7.373732,9.010052,5.0444,17.982514,13.352793,13.688067,13.821821,15.317179,16.485576,43.509785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,Tom Sample 10.wav,./Audio_Dataset/toms/Tom Sample 10.wav,469.67,1564.07,-526.057495,15.125813,14.260932,26.691828,10.059981,9.021091,...,-5.62146,-5.769227,-6.182073,28.643042,14.494631,17.842615,13.279404,16.241392,20.199279,40.605332
156,Tom Sample 12.wav,./Audio_Dataset/toms/Tom Sample 12.wav,429.62,1657.78,-468.365295,19.179003,16.861958,25.360548,11.881137,11.819502,...,-2.883872,-4.72994,-5.33417,27.207076,15.485106,17.586823,14.449008,16.067335,17.673519,38.6999
157,Tom Sample 5.wav,./Audio_Dataset/toms/Tom Sample 5.wav,758.22,2757.49,-482.770966,49.344788,27.762581,30.165827,16.573771,15.817742,...,-2.759053,-4.875147,-5.931608,24.591504,18.163808,15.165053,12.905551,13.673797,13.816295,43.79691
158,Tom Sample 4.wav,./Audio_Dataset/toms/Tom Sample 4.wav,691.61,2551.36,-491.714722,45.110367,25.816858,25.493029,12.545935,11.918398,...,-5.30627,-6.182038,-6.314246,23.852293,21.421241,15.184786,13.346316,14.775987,13.746777,43.258547


---
# **SAVE FEATURE FILES**


In [19]:
path = os.path.join(FEAT_FOLDER)
print("save features in " + path)

isExist = os.path.exists(path)
if not isExist:

   # Create a new directory because it does not exist
   os.makedirs(path)
   print("The new directory is created!")

print("save features to -> "+path + "feature.pkl")
with open(os.path.join(path, "feature.pkl"), 'wb') as fout:
   #pickle.dump(feat_dict, fout, protocol=pickle.HIGHEST_PROTOCOL)
   pickle.dump(feat_df, fout, protocol=pickle.HIGHEST_PROTOCOL)

save features in ./Features
save features to -> ./Featuresfeature.pkl
